In [1]:
from sklearn import datasets, utils
import tensorflow as tf
from sklearn.preprocessing import label_binarize
import numpy as np

raw_data_set = datasets.load_iris()
iris = np.column_stack([raw_data_set['data'], raw_data_set['target']])
iris = utils.shuffle(iris)
target_multiclass = label_binarize(iris[:, -1], classes=[0, 1, 2])

# Network Parameters
n_hidden_1 = 8
n_hidden_2 = 8
n_input = 4
n_classes = 3

# tf Graph input
X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, n_classes])

weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Create model
# Hidden fully connected layer with 8 neurons

layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
out_layer = tf.matmul(layer_2, weights['out']) + biases['out']

# Parameters
learning_rate = 0.001
training_epochs = 30
batch_size = 10
display_step = 1

# Define loss and optimizer

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Initializing  the variables

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(iris[:, -1][:100])/batch_size)
        train_x = iris[:100, :4]
        train_y = target_multiclass[:100]
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = train_x[i:i+batch_size], train_y[i:i+batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(out_layer)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: iris[100:150, :4], Y: target_multiclass[100:150]}))

Epoch: 0001 cost=81.494747925
Epoch: 0002 cost=74.904942703
Epoch: 0003 cost=68.512000275
Epoch: 0004 cost=62.291079712
Epoch: 0005 cost=56.234305954
Epoch: 0006 cost=50.329198837
Epoch: 0007 cost=44.559730530
Epoch: 0008 cost=38.907424736
Epoch: 0009 cost=33.352125168
Epoch: 0010 cost=27.872505379
Epoch: 0011 cost=22.446246433
Epoch: 0012 cost=17.049951172
Epoch: 0013 cost=11.658786964
Epoch: 0014 cost=6.250534534
Epoch: 0015 cost=1.549375266
Epoch: 0016 cost=0.101424005
Epoch: 0017 cost=0.005456123
Epoch: 0018 cost=0.002931477
Epoch: 0019 cost=0.003043045
Epoch: 0020 cost=0.003133118
Epoch: 0021 cost=0.003115915
Epoch: 0022 cost=0.003051914
Epoch: 0023 cost=0.002976017
Epoch: 0024 cost=0.002901963
Epoch: 0025 cost=0.002834182
Epoch: 0026 cost=0.002773595
Epoch: 0027 cost=0.002719971
Epoch: 0028 cost=0.002672676
Epoch: 0029 cost=0.002631049
Epoch: 0030 cost=0.002594427
Optimization Finished!
Accuracy: 0.92
